# Phase 4: Data Balancing for $t=2$ **without** Threshold Opt.
* Model: Stacking

## Setup

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Common imports
import numpy as np
import pandas as pd

from sklearn.model_selection import (
    cross_val_predict, cross_validate,
    RepeatedStratifiedKFold,
)

from imblearn.over_sampling import (
    RandomOverSampler, SMOTE, ADASYN, BorderlineSMOTE
)
from imblearn.under_sampling import (
    AllKNN,
    CondensedNearestNeighbour,
    EditedNearestNeighbours,
    InstanceHardnessThreshold,
    OneSidedSelection,
    RandomUnderSampler,
    NeighbourhoodCleaningRule,
    RepeatedEditedNearestNeighbours,
    TomekLinks,
)
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.pipeline import Pipeline
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier


## Data Loading

In [2]:
seed = 42
df = pd.read_csv(f"datasets/train_set_{seed}_t_2.csv")

Separate features and labels from the train-set and valid-set

In [3]:
X_80 = df.drop(['is_drop'], axis=1)
y_80 = df['is_drop'].copy()

X_80.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25406 entries, 0 to 25405
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index_o            25406 non-null  int64  
 1   year               25406 non-null  int64  
 2   semester           25406 non-null  int64  
 3   grade              25406 non-null  int64  
 4   sex                25406 non-null  object 
 5   gpa_last_seme      25406 non-null  float64
 6   credits_last_seme  25406 non-null  float64
 7   credits_tot        25406 non-null  float64
 8   n_seme             25406 non-null  int64  
 9   years_since        25406 non-null  int64  
 10  college            25406 non-null  object 
 11  adm_unit           25406 non-null  int64  
 12  nation             25406 non-null  int64  
 13  in_capa            25406 non-null  bool   
 14  leave              25406 non-null  bool   
dtypes: bool(2), float64(3), int64(8), object(2)
memory usage: 2.6+ MB


## Data Transformation

In [4]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

num_attrs = ['grade', 'gpa_last_seme', 'credits_last_seme', 'credits_tot', 'n_seme', 'years_since']
cat_attrs = ['semester', 'sex', 'adm_unit', 'nation', 'in_capa', 'college', 'leave']

full_pipeline = ColumnTransformer([
    ('num', StandardScaler(), num_attrs),
    ('cat', OneHotEncoder(), cat_attrs)
])

cat_pipeline = ColumnTransformer([
    ('num', StandardScaler(), num_attrs),
    ('cat', 'passthrough', cat_attrs)
])

# dataset of original test-set (80%)
X_t = full_pipeline.fit_transform(X_80)
X_80 = pd.DataFrame(X_t, columns=full_pipeline.get_feature_names_out())

cat_indices = [6, 7, 8, 9, 10, 11, 12]

# Typesetting
num_cols = ['num__grade', 'num__gpa_last_seme', 'num__credits_last_seme', 'num__credits_tot', 'num__n_seme', 'num__years_since']
cat_cols = list(set(X_80.columns) - set(num_cols))

X_80[num_cols] = X_80[num_cols].astype(float)
X_80[cat_cols] = X_80[cat_cols].astype(int)

## Model Training with Resampling

In [5]:
sampler_map = {
    "smote": SMOTE(random_state=0),
    "adasyn": ADASYN(
        sampling_strategy="auto",  # samples only the minority class
        random_state=0,  # for reproducibility
        n_neighbors=5,
    ),
    "cnn": CondensedNearestNeighbour(
        sampling_strategy="auto", random_state=0, n_neighbors=1, n_jobs=4
    ),
    "enn": EditedNearestNeighbours(
        sampling_strategy="auto", n_neighbors=3, kind_sel="all", n_jobs=4
    ),
    "tomek": TomekLinks(sampling_strategy="auto", n_jobs=4),
    "smoteenn": SMOTEENN(random_state=0),
}

Define the stacking model.

In [6]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_predict, cross_validate

cat_indices = [6, 7, 8, 9, 10, 11, 12]

estimators = [
    ('svm', SVC(C=10, gamma=0.1, random_state=42)),
    ('mlp', MLPClassifier(max_iter=10_000, hidden_layer_sizes=[10,10,10], random_state=42)),
    ('xgb', XGBClassifier(max_depth=4, n_estimators=70)),
    ('lgbm', LGBMClassifier(max_depth=8, n_estimators=100)),
    ('cat', CatBoostClassifier()),
]

In [7]:
stk = {}

for sampler in sampler_map.keys():
    print(sampler)
    X, y = sampler_map[sampler].fit_resample(X_80, y_80)

    # set up the classifier
    stk[sampler] = StackingClassifier(
        estimators=estimators, final_estimator=LogisticRegression(C=10)
    )

    stk[sampler].fit(X, y)


smote
[LightGBM] [Info] Number of positive: 24092, number of negative: 24092
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001056 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1566
[LightGBM] [Info] Number of data points in the train set: 48184, number of used features: 32
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Learning rate set to 0.053893
0:	learn: 0.6592519	total: 138ms	remaining: 2m 18s
1:	learn: 0.6308522	total: 144ms	remaining: 1m 11s
2:	learn: 0.6059099	total: 150ms	remaining: 49.8s
3:	learn: 0.5734645	total: 156ms	remaining: 38.9s
4:	learn: 0.5522312	total: 162ms	remaining: 32.3s
5:	learn: 0.5333092	total: 169ms	remaining: 28s
6:	learn: 0.5192811	total: 175ms	remaining: 24.8s
7:	learn: 0.5060834	total: 181ms	remaining: 22.5s
8:	learn: 0.4930892	total: 187ms	remaining: 20.6s
9:	learn: 0

## Model Evaluation

In [8]:
from sklearn.metrics import (
    precision_recall_curve, roc_curve, roc_auc_score, average_precision_score,
    recall_score, precision_score, f1_score, matthews_corrcoef
)
from pprint import pprint

# function to train models and evaluate the performance

def run_model(clf, X_test, y_test): 
    if (type(clf).__name__ == 'SVC'):
        y_score = clf.decision_function(X_test)
    else: y_score = clf.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_score)
    ap = average_precision_score(y_test, y_score)

    if (type(clf).__name__ == 'SVC'):
        y_pred = (y_score >= 0)
    else: y_pred = (y_score >= 0.5)
    recall_no = recall_score(y_test, y_pred)
    preci_no = precision_score(y_test, y_pred)
    f1_no = f1_score(y_test, y_pred)
    mcc_no = matthews_corrcoef(y_test, y_pred)

    precision, recall, thresholds = precision_recall_curve(y_test, y_score)

    # Maximizing F1-score
    f1_val = (2 * precision * recall) / (precision + recall)
    f1_val[np.isinf(f1_val)] = np.nan

    i = np.nanargmax(f1_val)
    best_thre = thresholds[i]

    y_pred = (y_score >= best_thre)
    recall_opt = recall_score(y_test, y_pred)
    preci_opt = precision_score(y_test, y_pred)
    f1_opt = f1_score(y_test, y_pred)
    mcc_opt = matthews_corrcoef(y_test, y_pred)


    res = {'ap': ap, 'auc': auc, 'recall': recall_no, 'precision': preci_no, 
           'f1': f1_no, 'mcc_no': mcc_no, 'recall(opt)': recall_opt, 
           'precision(opt)': preci_opt, 'f1(opt)': f1_opt, 'mcc(opt)': mcc_opt,
           'threshold': best_thre}
    print("model: ", type(clf).__name__)
    pprint(res)
    return res

In [9]:
import joblib

names = ['stk']
for name in names:
    model_dict = {
        'model': globals()[f'{name}'],
        #'result': globals()[f'res_{name}']
    }
    joblib.dump(model_dict, f'models/imbal_default_{name}_1_2.pkl')


## Final Test

In [10]:
test_set = pd.read_csv(f"datasets/test_set_{seed}_t_2.csv")

In [11]:
X_te = full_pipeline.transform(test_set)
y_te = test_set['is_drop'].copy()

X_te = pd.DataFrame(X_te, columns=full_pipeline.get_feature_names_out())

# Typesetting 
X_te[num_cols] = X_te[num_cols].astype(float)
X_te[cat_cols] = X_te[cat_cols].astype(int)

In [12]:
# function to train models and evaluate the performance

def test_model(clf, X_test, y_test): 
    if (type(clf).__name__ == 'SVC'):
        y_score = clf.decision_function(X_test)
    else: y_score = clf.predict_proba(X_test)[:, 1]

    auc = roc_auc_score(y_test, y_score)
    ap = average_precision_score(y_test, y_score)

    if (type(clf).__name__ == 'SVC'):
        y_pred = (y_score >= 0)
    else: y_pred = (y_score >= 0.5)
    recall_no = recall_score(y_test, y_pred)
    preci_no = precision_score(y_test, y_pred)
    f1_no = f1_score(y_test, y_pred)
    mcc_no = matthews_corrcoef(y_test, y_pred)

    res = {'ap': ap, 'auc': auc, 'recall': recall_no, 'precision': preci_no, 
           'f1': f1_no, 'mcc': mcc_no}
    print("model: ", type(clf).__name__)
    pprint(res, sort_dicts=False)
    return res

In [13]:
# now we train several models, with the different UnderSampling techniques and
# with cross-validation 

# save results

test_stk = {}

for sampler in sampler_map.keys():
    print(sampler)
    
    # set up the classifier
    test_stk[sampler] = test_model(stk[sampler], X_te, y_te)

    print()

smote
model:  StackingClassifier
{'ap': 0.4623869296809682,
 'auc': 0.8800232532734055,
 'recall': 0.4716981132075472,
 'precision': 0.4934210526315789,
 'f1': 0.48231511254019294,
 'mcc': 0.45585416037647175}

adasyn
model:  StackingClassifier
{'ap': 0.46324846667248865,
 'auc': 0.8935304399394479,
 'recall': 0.4779874213836478,
 'precision': 0.4550898203592814,
 'f1': 0.4662576687116564,
 'mcc': 0.4376000918591707}

cnn
model:  StackingClassifier
{'ap': 0.4540614203208241,
 'auc': 0.8969591272999672,
 'recall': 0.3742138364779874,
 'precision': 0.5748792270531401,
 'f1': 0.4533333333333333,
 'mcc': 0.44173821093816845}

enn
model:  StackingClassifier
{'ap': 0.45705673064005703,
 'auc': 0.8833005602634332,
 'recall': 0.4716981132075472,
 'precision': 0.4966887417218543,
 'f1': 0.4838709677419355,
 'mcc': 0.4576247091989063}

tomek
model:  StackingClassifier
{'ap': 0.48346942129441617,
 'auc': 0.9016071809437605,
 'recall': 0.34276729559748426,
 'precision': 0.6449704142011834,
 'f1': 